In [2]:
!pip install kafka-python
!pip install python-twitter
!pip install tweepy

In [3]:
import tweepy
import time
from kafka import KafkaConsumer, KafkaProducer

In [7]:
# Twitter API's Key

access_token = '811205561643515904-89kdYOal4whT4hoyzg5KMqpDu8Mx1fK'
access_token_secret = 'zMVNpFtHI9fVvtuP0LnomptaHhEQ7aKEWGwjV3iW504Zo'
consumer_key = 'obQRWkMGxeM5HTgigVM6uK91V'
consumer_secret = 'BTyRtsvKZLrWEOeOaF3nYSNd74i0LkRyNNumfn33a5xPKki8B1'

In [8]:
# Creating authentication object

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

In [9]:
# Setting your access token and secret

auth.set_access_token(access_token, access_token_secret)

In [10]:
# Creating the API object by passing in auth information

api = tweepy.API(auth)

In [11]:
# Formatting date and time

from datetime import datetime

def normalize_timestamp(time):
    mytime = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
    return mytime.strftime("%Y-%m-%d %H:%M:%S")

In [51]:
# KafkaProducer

producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
              api_version=(0,11,5))
topic_name = 'trump'

In [48]:
# Get data from Twitter

from pprint import pprint

def get_twitter_data():
    res = api.search_tweets("BBCI OR Sourav OR Ganguly")
    for i in res:
        record = ""
        record += str(i.user.id_str)
        record += ';'
        record += str(str(i.created_at))
        record += ';'
        record += str(i.user.followers_count)
        record += ';'
        record += str(i.user.location)
        record += ';'
        record += str(i.favorite_count)
        record += ';'
        record += str(i.retweet_count)
        record += ';'
        producer.send(topic_name, str.encode(record))

In [49]:
def periodic_work(interval):
    while True:
        get_twitter_data()
        # interval should be an integer, the number of seconds to wait
        time.sleep(interval)
        
periodic_work(60*0.1) # get data every couple of minutes

KafkaTimeoutError: KafkaTimeoutError: Failed to update metadata after 60.0 secs.